> Proszę zaimplementować:

1. Scalanie dwóch posortowanych list jednokierunkowych do jednej.

2. Algorytm sortowania list jednokierunkowych przez scalanie serii naturalnych.

3. Co się stanie, jeśli w powyższym algorytmie będziemy łączyć poprzednio posortowaną listę z kolejną, zamiast łączenia dwóch kolejnych list?

#### Uproszczona imlementacja struktury na potrzeby tego zadania

In [1]:
class Node:
    def __init__(self, val):
        self.val = val
        self.next = None


class LinkedList:
    def __init__(self, iterable=None):
        self.head = self.tail = None
        self._length = 0
        iterable and self.extend(iterable)
        
    def __len__(self):
        return self._length
        
    def __iter__(self):
        curr = self.head
        while curr:
            yield curr.val
            curr = curr.next
        
    def __str__(self):
        return ' -> '.join(map(lambda v: str(v), self))
    
    def append(self, val): 
        node = Node(val)
        if not self: self.head = self.tail = node
        else:
            self.tail.next = node
            self.tail = node
        self._length += 1

    def extend(self, values):
        for val in values:
            self.append(val)

## #1 Scalanie 2 posortowanych list (lepszy sposób - przepina węzły) (modyfikuje przekazane listy)

Posortowane listy są dane, więc nie musimy ich sortować. W kodzie testującym działanie, korzystam z wbudowanej w Pythona funkcji sortującej, ponieważ nie jest to częścią zadania, a ta funkcja nie stanowi elementu algorytmu, a jedynie służy do przygotowania danych wejściowych.
###### UWAGA
Algorytm ten "czyści" listy wejściowe, przepinając ich węzły do listy, będącej wynikiem łączenia. Dlatego w wynikach testu listy wejściowe są puste, ponieważ zostały one wyczyszczone przez algorytm łączenia list.

##### Funkcja testująca poprawność algorytmu

In [2]:
import random

def random_sorted_values():
    return sorted(random.randint(-100, 100) for _ in range(random.randint(0, 15)))

def test(samples=20):
    passed = 0

    for i in range(samples):
        vals1 = random_sorted_values()
        vals2 = random_sorted_values()
        expected = sorted(vals1 + vals2)

        ll1 = LinkedList(vals1)
        ll2 = LinkedList(vals2)
        res = merge_sorted(ll1, ll2)
        
        is_correct = list(res) == expected
        passed += is_correct

        print('List 1:', ll1)
        print('List 2:', ll2)
        print('Merged list:', res)
        print('Expected list:', LinkedList(expected))
        print(f'Test {"passed" if is_correct else "failed"}')
        print(f'Already passed-to-tested ratio: {passed}/{i+1}')
        print()
    
    print(f'Total tests passed: {passed}/{samples}')
    print(f'An algorithm is {"CORRECT" if passed == samples else "WRONG"}')

### Implementacja algorytmu

W poniższej implementacji przepinamy całe węzły ($ Node $) lub fragmenty list jednokierunkowych, więc potrzebna nam będzie metoda $ append\_nodes $, którą zaimplementujemy poniżej (poza klasą $ LinkedList $). Przyda nam się pewien sposób na odpinanie węzłów od przekazanych list, więc dodamy metodę $ popleft\_node $ do klasy $ LinkedList $.

In [3]:
def popleft_node(self):
    if not self:
        raise IndexError(f"pop from an empty {self.__class__.__name__}")
    removed_node = self.head
    if len(self) == 1:
        self.head = self.tail = None
    else:
        self.head = self.head.next
    removed_node.next = None
    self._length -= 1
    return removed_node


LinkedList.popleft_node = popleft_node

In [4]:
def merge_sorted(ll1, ll2):
    ll_res = LinkedList([None])  # Add a sentinel node
    ll_res._length = len(ll1) + len(ll2)
    
    while ll1 and ll2:
        if ll1.head.val < ll2.head.val:
            ll_res.tail.next = ll1.popleft_node()
        else:
            ll_res.tail.next = ll2.popleft_node()
        ll_res.tail = ll_res.tail.next
        
    # Link a remaining part to a result linked list
    if ll1:
        ll_res.tail.next = ll1.head
        ll1.head = ll1.tail = None
        ll1._length = 0
    else:
        ll_res.tail.next = ll2.head
        ll2.head = ll2.tail = None
        ll2._length = 0
        
    # Move a tail pointer to the last element
    while ll_res.tail.next:
        ll_res.tail = ll_res.tail.next
        
    # Remove a sentinel node
    ll_res.head = ll_res.head.next
        
    return ll_res

###### Kilka testów

In [5]:
test(100)

List 1: 
List 2: 
Merged list: -84 -> -82 -> -77 -> -58 -> -16 -> -4 -> 3 -> 24 -> 32 -> 32 -> 40 -> 43 -> 50 -> 72 -> 78 -> 80 -> 84 -> 88 -> 90
Expected list: -84 -> -82 -> -77 -> -58 -> -16 -> -4 -> 3 -> 24 -> 32 -> 32 -> 40 -> 43 -> 50 -> 72 -> 78 -> 80 -> 84 -> 88 -> 90
Test passed
Already passed-to-tested ratio: 1/1

List 1: 
List 2: 
Merged list: -94 -> -76 -> -54 -> -40 -> -22 -> 17 -> 33 -> 61
Expected list: -94 -> -76 -> -54 -> -40 -> -22 -> 17 -> 33 -> 61
Test passed
Already passed-to-tested ratio: 2/2

List 1: 
List 2: 
Merged list: -97 -> -90 -> -85 -> -77 -> -68 -> -47 -> -42 -> -30 -> -22 -> 5 -> 12 -> 17 -> 27 -> 35 -> 55 -> 68 -> 69 -> 69 -> 75 -> 85
Expected list: -97 -> -90 -> -85 -> -77 -> -68 -> -47 -> -42 -> -30 -> -22 -> 5 -> 12 -> 17 -> 27 -> 35 -> 55 -> 68 -> 69 -> 69 -> 75 -> 85
Test passed
Already passed-to-tested ratio: 3/3

List 1: 
List 2: 
Merged list: -99 -> -91 -> -72 -> -59 -> -58 -> -45 -> -17 -> -12 -> -8 -> 6 -> 8 -> 18 -> 20 -> 90
Expected list: -9

## #1 Scalanie 2 posortowanych list (gorszy sposób - nie przepina węzłów) (nie modyfikuje przekazanych list)

### Implementacja algorytmu

In [6]:
def merge_sorted(ll1, ll2):
	ll1_curr = ll1.head
	ll2_curr = ll2.head
	ll_res = LinkedList()

	while ll1_curr and ll2_curr:
		if ll1_curr.val < ll2_curr.val:
			ll_res.append(ll1_curr.val)
			ll1_curr = ll1_curr.next
		else:
			ll_res.append(ll2_curr.val)
			ll2_curr = ll2_curr.next

	while ll1_curr:
		ll_res.append(ll1_curr.val)
		ll1_curr = ll1_curr.next

	while ll2_curr:
		ll_res.append(ll2_curr.val)
		ll2_curr = ll2_curr.next

	return ll_res

###### Kilka testów

In [7]:
test(100)

List 1: -84 -> -63 -> -19 -> -10 -> 34 -> 41 -> 46 -> 48 -> 56 -> 77
List 2: -95 -> -88 -> -75 -> -7 -> 24 -> 43 -> 44 -> 69 -> 83 -> 90 -> 95
Merged list: -95 -> -88 -> -84 -> -75 -> -63 -> -19 -> -10 -> -7 -> 24 -> 34 -> 41 -> 43 -> 44 -> 46 -> 48 -> 56 -> 69 -> 77 -> 83 -> 90 -> 95
Expected list: -95 -> -88 -> -84 -> -75 -> -63 -> -19 -> -10 -> -7 -> 24 -> 34 -> 41 -> 43 -> 44 -> 46 -> 48 -> 56 -> 69 -> 77 -> 83 -> 90 -> 95
Test passed
Already passed-to-tested ratio: 1/1

List 1: -78 -> -78 -> -70 -> -39 -> -31 -> -16 -> 29 -> 54 -> 81 -> 97
List 2: -94 -> -87 -> -27
Merged list: -94 -> -87 -> -78 -> -78 -> -70 -> -39 -> -31 -> -27 -> -16 -> 29 -> 54 -> 81 -> 97
Expected list: -94 -> -87 -> -78 -> -78 -> -70 -> -39 -> -31 -> -27 -> -16 -> 29 -> 54 -> 81 -> 97
Test passed
Already passed-to-tested ratio: 2/2

List 1: -91 -> -36 -> 43
List 2: -88 -> -87 -> -67 -> -61 -> -51 -> -38 -> -22 -> -1 -> 35 -> 52 -> 78 -> 83 -> 86 -> 89
Merged list: -91 -> -88 -> -87 -> -67 -> -61 -> -51 -> -3

## #2 Merge Sort dla listy jednokierunkowej, scalający serie naturalne

Wyjaśnienie, czym jest seria naturalna, znajduje się w pliku Zadanie 1a. Implementacja klasy jest taka sama, jak wcześniej.

### Implementacja algorytmu

In [8]:
def merge_sort(ll):
    ll_head = ll.head
    
    while True:
        new_head = new_tail = None
        
        while True:
            # Cut off the first part from a linked list
            first_part = ll_head
            ll_head = cut_series(ll_head)
            
            if ll_head is None:
                # If a current linked list is empty after a cut and there are
                # no nodes in a new linked list, we have finished sorting
                if new_head is None:
                    ll.head = first_part
                    return
                # If we have cut the last part of a current linked list and
                # there are still some nodes in a new linked list, we have to
                # link a part cut to a new linked list.
                else:
                    new_tail.next = first_part
                    break
            
            # If the inner loop hasn't been broken yet, we can cut off
            # another part from a linked list
            second_part = ll_head
            ll_head = cut_series(ll_head)
            
            # As we have now two parts, we have to merge them together
            # and link to a new linked list
            merged_head, merged_tail = merged(first_part, second_part)
            
            if not new_head:
                new_head = merged_head
            else:
                new_tail.next = merged_head
            
            new_tail = merged_tail
            
        # If a current linked list was exhausted, replace it with a new linked list
        if ll_head is None:
            ll_head = new_head
            
            
def merged(ll1_curr, ll2_curr):
    res_head = res_tail = Node(None)  # Add a sentinel node to ease nodes linking
    
    while ll1_curr and ll2_curr:
        if ll1_curr.val < ll2_curr.val:
            res_tail.next = ll1_curr
            ll1_curr = ll1_curr.next
        else:
            res_tail.next = ll2_curr
            ll2_curr = ll2_curr.next
        res_tail = res_tail.next
        
    # Link the remaining nodes at the end of a result linked list
    if ll1_curr: res_tail.next = ll1_curr
    else: res_tail.next = ll2_curr
    
    # Move a tail pointer to the last node
    while res_tail.next:
        res_tail = res_tail.next
        
    # Remove a sentinel node
    res_head = res_head.next
        
    return res_head, res_tail


def cut_series(ll_head):
    if not ll_head: return None
    curr = ll_head
    while curr.next and curr.next.val >= curr.val:
        curr = curr.next
    # Return a pointer to the beginning of the second part after split
    remaining = curr.next
    curr.next = None
    return remaining

##### Funkcja testująca poprawność algorytmu

In [9]:
import random

def test(samples=20):
    passed = 0
    
    for i in range(samples):
        random_lst = [random.randint(-100, 100) for _ in range(random.randint(0, 40))]
        expected = sorted(random_lst)
        ll = LinkedList(random_lst)
        merge_sort(ll)
        result = list(ll)
        is_correct = expected == result
        passed += is_correct
        print('Input:', random_lst)
        print('Result:', result)
        print('Expected:', expected)
        print(f'Test {"PASSED" if is_correct else "FAILED"}')
        print(f'Already passed-to-tested ratio: {passed}/{i+1}')
        print()
        
    print(f'Total tests passed: {passed}/{samples}')
    print(f'An algorithm is {"CORRECT" if passed == samples else "WRONG"}')

###### Kilka testów

In [10]:
 test(100)

Input: [92, 35, -18, 26, -82, 78, -72, 100, 29, -89, -61, -43, -19, 52, 2, 11, 80, -97, -18, -16, -6, 52, -43, -82, 19, -89, -9, -85, -3, 62, 86, 59, 69, 17, 58, 62]
Result: [-97, -89, -89, -85, -82, -82, -72, -61, -43, -43, -19, -18, -18, -16, -9, -6, -3, 2, 11, 17, 19, 26, 29, 35, 52, 52, 58, 59, 62, 62, 69, 78, 80, 86, 92, 100]
Expected: [-97, -89, -89, -85, -82, -82, -72, -61, -43, -43, -19, -18, -18, -16, -9, -6, -3, 2, 11, 17, 19, 26, 29, 35, 52, 52, 58, 59, 62, 62, 69, 78, 80, 86, 92, 100]
Test PASSED
Already passed-to-tested ratio: 1/1

Input: [88, -44, 34, -75, -24, 59, 86, -77, -23, -91, -42, 51, -5, 73, -97, -41, -23, 57, -41]
Result: [-97, -91, -77, -75, -44, -42, -41, -41, -24, -23, -23, -5, 34, 51, 57, 59, 73, 86, 88]
Expected: [-97, -91, -77, -75, -44, -42, -41, -41, -24, -23, -23, -5, 34, 51, 57, 59, 73, 86, 88]
Test PASSED
Already passed-to-tested ratio: 2/2

Input: [63, -7, 13, 51, -79, 5, -71, 26, -4, -67, -50, -41, -73, 56, -21, 47, -7, 62, 56, 23, 44, -23, 62]
Resu

## #2 (Dodatkowo) 'Zwyczajny' Merge Sort dla listy jednokierunkowej

Implementacja klasy taka sama, jak wcześniej, więc nie wklejam jej poniżej.

Warto zaznaczyć, że w przeciwieństwie do tradycyjnej implementacji Merge Sorta (dla tablic), który nie modyfikuje przekazanej tablicy w miejscu, a jedynie zwraca nową tablicę, zawierającą posortowane wartości (oczywiście dlatego wymaga on dodatkowej pamięci - złożoność pamięciowa wynosi $ O(n) $), wersja Merge Sorta dla list jednokierunkowych nie wymaga dodatkowej pamięci i modyfikuje listy w miejscu (przepina odsyłacze).

### Implementacja algorytmu

In [11]:
def merge_sort(ll):
	ll.head = _merge_sort_recur(ll.head)


def _merge_sort_recur(begin_ptr):
	# If the 'll' part of a Linked List that is being sorted has no more than
	# 1 element, return this part
	if not begin_ptr or not begin_ptr.next: return begin_ptr

	left_ptr, right_ptr = _split(begin_ptr)
	return _merge(_merge_sort_recur(left_ptr), _merge_sort_recur(right_ptr))


def _split(begin_ptr):
	# Find a place to make a cut (split current 'll' part into to halves)
	cut_ptr = begin_ptr
	end_ptr = begin_ptr.next

	while end_ptr:
		end_ptr = end_ptr.next
		if end_ptr:
			end_ptr = end_ptr.next
			cut_ptr = cut_ptr.next

	# Perform a cutting operation (split into the left and the right part)
	left_ptr = begin_ptr
	right_ptr = cut_ptr.next
	cut_ptr.next = None  # Unlink the right part from the left part

	return left_ptr, right_ptr


def _merge(left_ptr, right_ptr):
	if not left_ptr: return right_ptr
	if not right_ptr: return left_ptr

	if left_ptr.val < right_ptr.val:
		left_ptr.next = _merge(left_ptr.next, right_ptr)
		return left_ptr
	else:
		right_ptr.next = _merge(left_ptr, right_ptr.next)
		return right_ptr

###### Kilka testów

In [12]:
test(100)

Input: [11, 71, 26, 32, -96, -52, -86, 16, 48, -33, -56, 8, 24, 79, -75, -76, -64, 2, 93]
Result: [-96, -86, -76, -75, -64, -56, -52, -33, 2, 8, 11, 16, 24, 26, 32, 48, 71, 79, 93]
Expected: [-96, -86, -76, -75, -64, -56, -52, -33, 2, 8, 11, 16, 24, 26, 32, 48, 71, 79, 93]
Test PASSED
Already passed-to-tested ratio: 1/1

Input: [-100, -48, -80, 82, 22, 94, -5]
Result: [-100, -80, -48, -5, 22, 82, 94]
Expected: [-100, -80, -48, -5, 22, 82, 94]
Test PASSED
Already passed-to-tested ratio: 2/2

Input: [-41, -44, 27, 46, 12, -23, 51, -79, 12, -27, -16, 83, 70, 4, -37, -30, 41, -82, -72, -91, -71, 10, -16, 13, 83, -57, -53, -52, -65, 76, 29, -71]
Result: [-91, -82, -79, -72, -71, -71, -65, -57, -53, -52, -44, -41, -37, -30, -27, -23, -16, -16, 4, 10, 12, 12, 13, 27, 29, 41, 46, 51, 70, 76, 83, 83]
Expected: [-91, -82, -79, -72, -71, -71, -65, -57, -53, -52, -44, -41, -37, -30, -27, -23, -16, -16, 4, 10, 12, 12, 13, 27, 29, 41, 46, 51, 70, 76, 83, 83]
Test PASSED
Already passed-to-tested rati

## #2 (Dodatkowo) 'Zwyczajny' Merge Sort niemodyfikujący listy w miejscu

Implementacja klasy taka sama, jak wcześniej, więc nie wklejam jej poniżej.

### Implementacja algorytmu

In [13]:
def merge_sorted(ll):
	if len(ll) <= 1: return ll

	left_ll, right_ll = _split(ll)
	return _merged(merge_sorted(left_ll), merge_sorted(right_ll))


def _split(ll):
	mid_idx = len(ll) // 2
	left_ll = LinkedList()
	right_ll = LinkedList()

	curr = ll.head
	for _ in range(mid_idx):
		left_ll.append(curr.val)
		curr = curr.next
	
	while curr:
		right_ll.append(curr.val)
		curr = curr.next
	
	return left_ll, right_ll


def _merged(left_ll, right_ll):
	result = LinkedList()
	left_curr = left_ll.head
	right_curr = right_ll.head

	while left_curr and right_curr:
		if left_curr.val < right_curr.val:
			result.append(left_curr.val)
			left_curr = left_curr.next
		else:
			result.append(right_curr.val)
			right_curr = right_curr.next

	while left_curr:
		result.append(left_curr.val)
		left_curr = left_curr.next

	while right_curr:
		result.append(right_curr.val)
		right_curr = right_curr.next

	return result

##### Funkcja testująca poprawność algorytmu

In [14]:
import random

def test(samples=20):
    passed = 0
    
    for i in range(samples):
        random_lst = [random.randint(-100, 100) for _ in range(random.randint(0, 40))]
        expected = sorted(random_lst)
        ll = LinkedList(random_lst)
        result = list(merge_sorted(ll))
        is_correct = expected == result
        passed += is_correct
        print('Input:', random_lst)
        print('Result:', result)
        print('Expected:', expected)
        print(f'Test {"PASSED" if is_correct else "FAILED"}')
        print(f'Already passed-to-tested ratio: {passed}/{i+1}')
        print()
        
    print(f'Total tests passed: {passed}/{samples}')
    print(f'An algorithm is {"CORRECT" if passed == samples else "WRONG"}')

###### Kilka testów

In [15]:
test(100)

Input: [-74, -34, -59, 21, 34, 41, -11, -49, -68, -37, -47, 95, -63, 10]
Result: [-74, -68, -63, -59, -49, -47, -37, -34, -11, 10, 21, 34, 41, 95]
Expected: [-74, -68, -63, -59, -49, -47, -37, -34, -11, 10, 21, 34, 41, 95]
Test PASSED
Already passed-to-tested ratio: 1/1

Input: [31, -69, 46, 72, 28, 46, 62, 20, -93, 46, 50, -37, 90, -75, -18, 21, -76, 47, -93, 32, -96, 40, 65, 51, -77, 69, 43, 19, 83, -31, -19, -56, 18, -15]
Result: [-96, -93, -93, -77, -76, -75, -69, -56, -37, -31, -19, -18, -15, 18, 19, 20, 21, 28, 31, 32, 40, 43, 46, 46, 46, 47, 50, 51, 62, 65, 69, 72, 83, 90]
Expected: [-96, -93, -93, -77, -76, -75, -69, -56, -37, -31, -19, -18, -15, 18, 19, 20, 21, 28, 31, 32, 40, 43, 46, 46, 46, 47, 50, 51, 62, 65, 69, 72, 83, 90]
Test PASSED
Already passed-to-tested ratio: 2/2

Input: [83, 78, 82, 88, -35, -63, -27, -7, -73, -44, -27, 21, -25, -47, -12, -65, -79, -19, -21, 22, -41, 33, 10, -73, 90, 86, -96, -74, 48, -70, 6, -24, -81, 57, -4, 18, 25, 80, -36]
Result: [-96, -81, -

Input: [38, 40, -65, -35, 93, -39, -53, 2, -22, -24, -74, -61, 26, -70, 70, -55, -63, 59, 22, -60, -66, -18, 89, 8, -96, 5, -56, -54, -13]
Result: [-96, -74, -70, -66, -65, -63, -61, -60, -56, -55, -54, -53, -39, -35, -24, -22, -18, -13, 2, 5, 8, 22, 26, 38, 40, 59, 70, 89, 93]
Expected: [-96, -74, -70, -66, -65, -63, -61, -60, -56, -55, -54, -53, -39, -35, -24, -22, -18, -13, 2, 5, 8, 22, 26, 38, 40, 59, 70, 89, 93]
Test PASSED
Already passed-to-tested ratio: 76/76

Input: [-39, 78, 81, 55, 19, 77, 0, 23, -45, -20, -5, 77, -96, 4, -73, 98, -17, -18, -85, 39, 89, 60, 14, -86, 73, -20, 67, -75]
Result: [-96, -86, -85, -75, -73, -45, -39, -20, -20, -18, -17, -5, 0, 4, 14, 19, 23, 39, 55, 60, 67, 73, 77, 77, 78, 81, 89, 98]
Expected: [-96, -86, -85, -75, -73, -45, -39, -20, -20, -18, -17, -5, 0, 4, 14, 19, 23, 39, 55, 60, 67, 73, 77, 77, 78, 81, 89, 98]
Test PASSED
Already passed-to-tested ratio: 77/77

Input: [80, 14, 87, 50, 75, 39, 15, 3, -10, -65, -84, 50, -78, -90, 93, 54, -40, 44, -

## #3 Nie wiem, co autor miał na myśli.

Jeżeli fragment 'w powyższym algorytmie' odnosi się do algorytmu scalania z pkt. 1., a nie do algorytmu sortowania z pkt. 2. (jak sugeruje polecenie), a poprzez 'poprzednio posortowaną listę' rozumiemy listę jednokierunkową, która została posortowana (np. przy pomocy algorytmu z pkt. 2.), to wciąż fragment polecenia 'zamiast łączenia dwóch kolejnych list' jest pozbawiony sensu, bo nie wiadomo, co oznaczają dwie kolejne listy (skąd mamy wiąć te 2 KOLEJNE listy).

Jedyna sensowna interpretacja, jaka przychodzi mi do głowy, to modyfikacja algorytmu z punktu 1. tak, aby łączył on listę posortowaną z listą nieposortowaną, więc poniżej znajduje się jego implementacja.

W poniższej implementacji ponownie korzystam z wcześniej umieszczonej w tym pliku deklaracji klasy LinkedList. W tym przykładzie przydatne będą jeszcze metody $ insertafter $, $ popleft $ oraz $ appendleft $, które zaimplementujemy poniżej (poza deklaracją klasy).

### Implementacja algorytmu

In [16]:
def insertafter(self, prev_node, val):
    node = Node(val)
    node.next = prev_node.next
    prev_node.next = node
    self._length += 1
    
def popleft(self):
    if not self:
        raise IndexError(f"pop from empty {self.__class__.__name__}")
    removed = self.head.val
    if len(self) == 1:
        self.head = self.tail = None
    else:
        self.head = self.head.next
    self._length -= 1
    return removed

def appendleft(self, val):
    node = Node(val)
    if not self:
        self.head = self.tail = node
    else:
        node.next = self.head
        self.head = node
    self._length += 1
    
    
LinkedList.insertafter = insertafter
LinkedList.appendleft = appendleft
LinkedList.popleft = popleft

In [17]:
def merge_sorted_and_unsorted(sorted_ll, unsorted_ll):
    if sorted_ll:
        result_ll = LinkedList(sorted_ll)
    elif unsorted_ll:
        result_ll = LinkedList([unsorted_ll.popleft()])
    else:
        return LinkedList()
    
    while unsorted_ll:
        val = unsorted_ll.popleft()
        
        if val >= result_ll.tail.val:
            result_ll.append(val)
        elif val <= result_ll.head.val:
            result_ll.appendleft(val)
        else:
            curr = result_ll.head
            while val > curr.next.val:
                curr = curr.next
            result_ll.insertafter(curr, val)
            
    return result_ll

##### Funkcja testująca poprawność algorytmu

In [18]:
import random

def random_values():
    return [random.randint(-100, 100) for _ in range(random.randint(0, 15))]

def test(samples=20):
    passed = 0

    for i in range(samples):
        vals1 = sorted(random_values())
        vals2 = random_values()
        expected = sorted(vals1 + vals2)

        ll1 = LinkedList(vals1)
        ll2 = LinkedList(vals2)
        res = merge_sorted_and_unsorted(ll1, ll2)
        
        is_correct = list(res) == expected
        passed += is_correct

        print('List 1:', ll1)
        print('List 2:', ll2)
        print('Merged list:', res)
        print('Expected list:', LinkedList(expected))
        print(f'Test {"passed" if is_correct else "failed"}')
        print(f'Already passed-to-tested ratio: {passed}/{i+1}')
        print()
    
    print(f'Total tests passed: {passed}/{samples}')
    print(f'An algorithm is {"CORRECT" if passed == samples else "WRONG"}')

###### Kilka testów

In [19]:
test(100)

List 1: 
List 2: 
Merged list: -71 -> -57 -> -6 -> 4 -> 25 -> 29 -> 31 -> 32 -> 65 -> 73
Expected list: -71 -> -57 -> -6 -> 4 -> 25 -> 29 -> 31 -> 32 -> 65 -> 73
Test passed
Already passed-to-tested ratio: 1/1

List 1: -99 -> -90 -> -80 -> -37 -> 1 -> 26 -> 38 -> 81
List 2: 
Merged list: -99 -> -90 -> -80 -> -37 -> 1 -> 26 -> 38 -> 81
Expected list: -99 -> -90 -> -80 -> -37 -> 1 -> 26 -> 38 -> 81
Test passed
Already passed-to-tested ratio: 2/2

List 1: -69 -> -62 -> -12 -> 90 -> 98
List 2: 
Merged list: -69 -> -62 -> -61 -> -12 -> 90 -> 98
Expected list: -69 -> -62 -> -61 -> -12 -> 90 -> 98
Test passed
Already passed-to-tested ratio: 3/3

List 1: -94 -> -85 -> -62 -> -56 -> -56 -> -45 -> -43 -> -37 -> 40 -> 64 -> 89
List 2: 
Merged list: -94 -> -85 -> -72 -> -66 -> -62 -> -57 -> -56 -> -56 -> -45 -> -43 -> -43 -> -37 -> -32 -> -2 -> 20 -> 40 -> 64 -> 89
Expected list: -94 -> -85 -> -72 -> -66 -> -62 -> -57 -> -56 -> -56 -> -45 -> -43 -> -43 -> -37 -> -32 -> -2 -> 20 -> 40 -> 64 -> 89
T

List 2: 
Merged list: -90 -> -87 -> -85 -> -73 -> -44 -> -21 -> 4 -> 4 -> 11 -> 17 -> 20 -> 61 -> 73 -> 88
Expected list: -90 -> -87 -> -85 -> -73 -> -44 -> -21 -> 4 -> 4 -> 11 -> 17 -> 20 -> 61 -> 73 -> 88
Test passed
Already passed-to-tested ratio: 46/46

List 1: -82
List 2: 
Merged list: -82 -> -58
Expected list: -82 -> -58
Test passed
Already passed-to-tested ratio: 47/47

List 1: -85 -> -52 -> -8 -> 35 -> 76 -> 92 -> 95 -> 98
List 2: 
Merged list: -85 -> -52 -> -8 -> 35 -> 66 -> 76 -> 81 -> 92 -> 95 -> 98
Expected list: -85 -> -52 -> -8 -> 35 -> 66 -> 76 -> 81 -> 92 -> 95 -> 98
Test passed
Already passed-to-tested ratio: 48/48

List 1: -19 -> 28 -> 82
List 2: 
Merged list: -95 -> -93 -> -70 -> -65 -> -47 -> -46 -> -44 -> -26 -> -19 -> -16 -> 16 -> 27 -> 28 -> 62 -> 64 -> 74 -> 82
Expected list: -95 -> -93 -> -70 -> -65 -> -47 -> -46 -> -44 -> -26 -> -19 -> -16 -> 16 -> 27 -> 28 -> 62 -> 64 -> 74 -> 82
Test passed
Already passed-to-tested ratio: 49/49

List 1: -62 -> -31 -> -7 -> 2